# PEFT Project
simple project to demonstrate Parameter-Efficient Fine-Tunning Model using PEFT Library

Install needed packages

In [20]:
%pip install --index-url https://download.pytorch.org/whl/cu128 torch torchvision torchaudio
%pip install transformers peft scikit-learn pandas huggingface_hub datasets hf_xet evaluate ipywidgets

Looking in indexes: https://download.pytorch.org/whl/cu128
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# import needed classes

In [21]:
import torch
from transformers import DistilBertForSequenceClassification,DistilBertTokenizer,DataCollatorWithPadding,TrainingArguments,Trainer,pipeline
from datasets import load_dataset
import evaluate
import numpy as np
import pandas as pd
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification

## Helper functions

In [22]:
def get_bert_model_and_tokenizer():
    model_name = 'distilbert/distilbert-base-uncased'
    device = "cuda" if torch.cuda.is_available() else "cpu"
    base_model = DistilBertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=6,
        id2label={0:"sadness",1:"joy",2:"love",3:"anger",4:"fear",5:"surprise"},
        label2id={"sadness":0,"joy":1,"love":2,"anger":3,"fear":4,"surprise":5}
    )
    base_model = base_model.to(device)
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    return base_model, tokenizer

## Load pre-trained model and tokenizer

In [23]:
# load pre_trained model
base_model, base_tokenizer = get_bert_model_and_tokenizer()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Datasets

In [24]:
# load dataset
dataset_name = "dair-ai/emotion"  
dataset = load_dataset(dataset_name) 

# define tokenizer function
def tokenize_fun(dataset):
    return base_tokenizer(dataset["text"], padding="max_length", truncation=True)

# define datasets
train_dataset = dataset["train"].map(tokenize_fun, batched=True) # limit the dataset for fast training
test_dataset = dataset["test"].map(tokenize_fun, batched=True) # limit the dataset for fast evaluation
print(pd.DataFrame(dataset["train"]))
print(pd.DataFrame(dataset["test"]))

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

                                                    text  label
0                                i didnt feel humiliated      0
1      i can go from feeling so hopeless to so damned...      0
2       im grabbing a minute to post i feel greedy wrong      3
3      i am ever feeling nostalgic about the fireplac...      2
4                                   i am feeling grouchy      3
...                                                  ...    ...
15995  i just had a very brief time in the beanbag an...      0
15996  i am now turning and i feel pathetic that i am...      0
15997                     i feel strong and good overall      1
15998  i feel like this was such a rude comment and i...      3
15999  i know a lot but i feel so stupid because i ca...      0

[16000 rows x 2 columns]
                                                   text  label
0     im feeling rather rotten so im not very ambiti...      0
1             im updating my blog because i feel shitty      0
2     i never mak

## manually test model

In [25]:
tests = [
    "I feel little rainy ",
    "there are many definitions of 'time waster",
    "movie has good music"
]

base_model_pipeline = pipeline("text-classification", model=base_model, tokenizer=base_tokenizer, top_k=1)
for test in tests:
    print(f"base model test with '{test}' result => {base_model_pipeline(test)}")

Device set to use cuda:0


base model test with 'I feel little rainy ' result => [[{'label': 'surprise', 'score': 0.18865442276000977}]]
base model test with 'there are many definitions of 'time waster' result => [[{'label': 'surprise', 'score': 0.1885065734386444}]]
base model test with 'movie has good music' result => [[{'label': 'surprise', 'score': 0.18810170888900757}]]


## Evaluate pre-trained model

In [26]:
#define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# test pre-trained model against dataset and note total loss
data_collator = DataCollatorWithPadding(tokenizer=base_tokenizer)
accuracy = evaluate.load("accuracy")

# start evaluation 
original_model_eval_args = TrainingArguments(
    output_dir="original_eval_out",
    do_train=False,
    do_eval=True,
    per_device_eval_batch_size=32,
    dataloader_num_workers=0,
    fp16=torch.cuda.is_available(),
    report_to=[]
)

original_trainer = Trainer(
    model=base_model,
    args=original_model_eval_args,
    eval_dataset=test_dataset,
    tokenizer=base_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
original_eval_metrics = original_trainer.evaluate()
print(original_eval_metrics)

C:\Users\engmu\AppData\Local\Temp\ipykernel_23296\2924241023.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  original_trainer = Trainer(


{'eval_loss': 1.8177480697631836, 'eval_model_preparation_time': 0.0023, 'eval_accuracy': 0.0345, 'eval_runtime': 7.4765, 'eval_samples_per_second': 267.505, 'eval_steps_per_second': 8.426}


## Create PEFT model from pre-trained one

In [27]:
# create peft model
print(base_model)
peft_model_name = 'distilbert-mustafa-uncased';
config = LoraConfig(
    target_modules=["v_lin"],
    modules_to_save=["pre_classifier","classifier"]
)
peft_base_model, peft_tokenizer = get_bert_model_and_tokenizer()
peft_model = get_peft_model(peft_base_model,config)
peft_model.print_trainable_parameters()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 668,934 || all params: 67,627,020 || trainable%: 0.9892


## Train PEFT Model on train dataset

In [28]:
# train the model against dataset
peft_model_train_args = TrainingArguments(
    output_dir="peft_train_out",
    do_train=True,
    do_eval=False,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    dataloader_num_workers=0,
    fp16=torch.cuda.is_available(),
    report_to=[]
)

peft_trainer = Trainer(
    model=peft_model,
    args=original_model_eval_args,
    train_dataset=train_dataset,
    tokenizer=peft_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
peft_model_metrics = peft_trainer.train()
print(peft_model_metrics)
peft_model.save_pretrained("distilbert-mustafa-uncased")

C:\Users\engmu\AppData\Local\Temp\ipykernel_23296\1297619248.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


Step,Training Loss
500,1.397800
1000,1.117100
1500,0.944300
2000,0.833700
2500,0.741700
3000,0.685300
3500,0.638500
4000,0.652700
4500,0.599000
5000,0.588200


TrainOutput(global_step=6000, training_loss=0.7816309356689453, metrics={'train_runtime': 477.8163, 'train_samples_per_second': 100.457, 'train_steps_per_second': 12.557, 'total_flos': 6457527042048000.0, 'train_loss': 0.7816309356689453, 'epoch': 3.0})


## Test PEFT Model

In [29]:
peft_model = AutoPeftModelForSequenceClassification.from_pretrained(
    peft_model_name,
    num_labels=6,
    id2label={0:"sadness",1:"joy",2:"love",3:"anger",4:"fear",5:"surprise"},
    label2id={"sadness":0,"joy":1,"love":2,"anger":3,"fear":4,"surprise":5}
)
peft_model_eval_args = TrainingArguments(
    output_dir="peft_eval_out",
    do_train=False,
    do_eval=True,
    per_device_eval_batch_size=32,
    dataloader_num_workers=0,
    fp16=torch.cuda.is_available(),
    report_to=[]
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_model_eval_args,
    eval_dataset=test_dataset,
    tokenizer=base_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
peft_metrics = peft_trainer.evaluate()
print(peft_metrics)
print(f"peft result: original model eval loss {original_eval_metrics["eval_loss"]} => peft model eval loss {peft_metrics["eval_loss"]}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\engmu\AppData\Local\Temp\ipykernel_23296\1255388915.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


{'eval_loss': 0.5353090763092041, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.8155, 'eval_runtime': 7.6874, 'eval_samples_per_second': 260.167, 'eval_steps_per_second': 8.195}
peft result: original model eval loss 1.8177480697631836 => peft model eval loss 0.5353090763092041


## manually test both original model and PEFT model

In [30]:
tests = [
    "I feel little rainy ",
    "there are many definitions of 'time waster",
    "movie has good music"
]

peft_model_pipeline = pipeline("text-classification", model=peft_model,tokenizer=base_tokenizer, top_k=1)
for test in tests:
    print(f"base model test with '{test}' result => {base_model_pipeline(test)}")
    print(f"peft model test with '{test}' result => {peft_model_pipeline(test)}")

Device set to use cuda:0


base model test with 'I feel little rainy ' result => [[{'label': 'surprise', 'score': 0.18865975737571716}]]
peft model test with 'I feel little rainy ' result => [[{'label': 'sadness', 'score': 0.8379078507423401}]]
base model test with 'there are many definitions of 'time waster' result => [[{'label': 'surprise', 'score': 0.18853044509887695}]]
peft model test with 'there are many definitions of 'time waster' result => [[{'label': 'sadness', 'score': 0.5731086730957031}]]
base model test with 'movie has good music' result => [[{'label': 'surprise', 'score': 0.18811479210853577}]]
peft model test with 'movie has good music' result => [[{'label': 'joy', 'score': 0.9562808275222778}]]
